Используем данные соревнования [Articles Sharing and Reading from CI&T Deskdrop](https://www.kaggle.com/gspmoreira/articles-sharing-reading-from-cit-deskdrop). Основано на материалах Елены Кантонистовой.

In [1]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

Данные содержат набор логов за 12 месяцев (с марта 2016 по февраль 2017) с CI&T's Internal Communication platform (DeskDrop). Он содержит около 73 тыс. зарегистрированных взаимодействий пользователей с более чем 3 тыс. общедоступных статей, опубликованных на платформе.

Первый файл содержит информацию о статьях, опубликованных на платформе. У каждой статьи есть дата публикации (временная метка), исходный URL-адрес, заголовок, содержание в виде обычного текста, язык статьи (португальский: pt или английский: en) и информация о пользователе, который поделился статьей (авторе).

Для данной отметки времени существует два возможных типа событий:
- CONTENT SHARED: статья была опубликована на платформе и доступна для пользователей.
- CONTENT REMOVED: статья была удалена с платформы и недоступна для дальнейших рекомендаций.

Для простоты мы рассматриваем здесь только тип события «CONTENT SHARED».

In [2]:
articles_df = pd.read_csv('shared_articles.csv')
articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']
articles_df.head(5)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en
5,1459194522,CONTENT SHARED,-2826566343807132236,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.coindesk.com/ieee-blockchain-oxford...,IEEE to Talk Blockchain at Cloud Computing Oxf...,One of the largest and oldest organizations fo...,en


Во втором файле находятся логи взаимодействий пользователей с доступными статьями. Его можно объединить с предыдущим по колонке `contentId`. 

Значения `eventType`:

- **VIEW**: Пользователь открыл статью. 
- **LIKE**: Пользователь лайкнул статью. 
- **COMMENT CREATED**: Пользователь оставил комментарий под статьей. 
- **FOLLOW**: Пользователь включил уведомления о каждом новом комментарии под статьей. 
- **BOOKMARK**: Пользователь сделал закладку на статью (сохранил, чтобы иметь возможность быстро к ней вернуться).

In [3]:
interactions_df = pd.read_csv('users_interactions.csv')
interactions_df.head(10)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN
5,1465413742,VIEW,310515487419366995,-8763398617720485024,1395789369402380392,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,MG,BR
6,1465415950,VIEW,-8864073373672512525,3609194402293569455,1143207167886864524,NaN,NaN,NaN
7,1465415066,VIEW,-1492913151930215984,4254153380739593270,8743229464706506141,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR
8,1465413762,VIEW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
9,1465413771,VIEW,3064370296170038610,3609194402293569455,1143207167886864524,NaN,NaN,NaN


# Предобработка данных

Поскольку существуют разные типы взаимодействий, мы задаем им вес, предполагая, что, например, комментарий в статье указывает на более высокий интерес пользователя к элементу, чем лайк или простое представление.

In [4]:
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

interactions_df['eventStrength'] = interactions_df['eventType'].apply(lambda x: event_type_strength[x])

У рекомендательных систем есть проблема, известная как **user cold-start**, при которой сложно предоставить персонализированные рекомендации для пользователей, у которых нет или очень мало потребленных элементов, из-за отсутствия информации для моделирования их предпочтений.

По этой причине мы сохраняем в наборе данных только пользователей, у которых было как минимум 5 взаимодействий.

In [5]:
users_interactions_count_df = interactions_df.groupby(
    ['personId', 'contentId']).size().groupby('personId').size()
print('# users: %d' % len(users_interactions_count_df))
users_with_enough_interactions_df = users_interactions_count_df[
    users_interactions_count_df >= 5].reset_index()[['personId']]
print('# users with at least 5 interactions: %d' %
      len(users_with_enough_interactions_df))

# users: 1895
# users with at least 5 interactions: 1140


In [6]:
print('# of interactions: %d' % len(interactions_df))
interactions_from_selected_users_df = interactions_df.merge(
    users_with_enough_interactions_df,
    how='right',
    left_on='personId',
    right_on='personId')
print('# of interactions from users with at least 5 interactions: %d' %
      len(interactions_from_selected_users_df))

# of interactions: 72312
# of interactions from users with at least 5 interactions: 69868


В Deskdrop пользователям разрешено просматривать статью много раз и взаимодействовать с ними по-разному (например, ставить лайк или комментировать). Таким образом, чтобы смоделировать интерес пользователя к данной статье, мы агрегируем все взаимодействия, которые пользователь выполнил с элементом, по взвешенной сумме силы типа взаимодействия и применяем преобразование журнала для сглаживания распределения.

In [11]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId']).eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index().set_index(['personId', 'contentId'])
)
interactions_full_df['last_timestamp'] = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId'])['timestamp'].last()
)
        
interactions_full_df = interactions_full_df.reset_index()
interactions_full_df.head(20)
print('# of unique user/item interactions: %d' % len(interactions_full_df))
interactions_full_df.head(10)

# of unique user/item interactions: 39106


,personId,contentId,eventStrength,last_timestamp
0,-9223121837663643404,-8949113594875411859,1.000000,1462452127
1,-9223121837663643404,-8377626164558006982,1.000000,1473938707
2,-9223121837663643404,-8208801367848627943,1.000000,1469706702
3,-9223121837663643404,-8187220755213888616,1.000000,1467823897
4,-9223121837663643404,-7423191370472335463,3.169925,1479290925
5,-9223121837663643404,-7331393944609614247,1.000000,1463486417
6,-9223121837663643404,-6872546942144599345,1.000000,1487677800
7,-9223121837663643404,-6728844082024523434,1.000000,1485252387
8,-9223121837663643404,-6590819806697898649,1.000000,1485252385
9,-9223121837663643404,-6558712014192834002,1.584963,1464968564


Разобьём выборку на обучение и контроль по времени.

In [12]:
from sklearn.model_selection import train_test_split

split_ts = 1475519530
interactions_train_df = interactions_full_df.loc[interactions_full_df.last_timestamp < split_ts].copy()
interactions_test_df = interactions_full_df.loc[interactions_full_df.last_timestamp >= split_ts].copy()

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

interactions_train_df

# interactions on Train set: 29329
# interactions on Test set: 9777


,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.0,1470395911
2,-1007001694607905623,-793729620925729327,1.0,1472834892
6,-1032019229384696495,-1006791494035379303,1.0,1469129122
7,-1032019229384696495,-1039912738963181810,1.0,1459376415
8,-1032019229384696495,-1081723567492738167,2.0,1464054093
...,...,...,...,...
39099,997469202936578234,9112765177685685246,2.0,1472479493
39100,998688566268269815,-1255189867397298842,1.0,1474567164
39101,998688566268269815,-401664538366009049,1.0,1474567449
39103,998688566268269815,6881796783400625893,1.0,1474567675


Также разбить данные можно не опираясь на временной фактор:
```python
interactions_train_df, interactions_test_df = train_test_split(interactions_full_df,
                                   stratify=interactions_full_df['personId'], 
                                   test_size=0.25,
                                   random_state=42)

```
Как повлият такое разбиение на итоговый результат?

Для удобства подсчёта качества запишем данные в формате, где строка соответствует пользователю, а столбцы будут истинными метками и предсказанями в виде списков.

In [13]:
interactions = (
    interactions_train_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)

interactions['true_test'] = (
    interactions_test_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)

# заполнение пропусков пустыми списками
interactions.loc[pd.isnull(interactions.true_test), 'true_test'] = [
    list() for x in range(len(interactions.loc[pd.isnull(interactions.true_test), 'true_test']))]

interactions.head(1)

,true_train,true_test
personId,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72..."


## Часть 1: Baseline (модель по популярности)

Самой простой моделью рекомендаций (при этом достаточно сильной!) является модель, которая рекомендует наиболее популярные предметы. 

Реализуем её. Давайте считать, что рекомендуем мы по 10 материалов (такое ограничение на размер блока на сайте).

Посчитаем популярность каждой статьи, как сумму всех "оценок" взаимодействий с ней. Отсортируем материалы по их популярности.

In [14]:
# popular_content = 
####### Здесь ваш код ##########
popular_content = (
    interactions_train_df
    .groupby('contentId')
    .eventStrength.sum().reset_index()
    .sort_values('eventStrength', ascending=False)
    .contentId.values
)
################################

In [15]:
print(articles_df.loc[articles_df.contentId == popular_content[2]]['title'].values)

["Ray Kurzweil: The world isn't getting worse - our information is getting better"]


In [16]:
print(articles_df.loc[articles_df.contentId == popular_content[2363]]['title'].values)

['Sunset Blvd. (1950)']


Теперь необходимо сделать предсказания для каждого пользователя. Не забываем, что надо рекомендовать то, что пользователь ещё не читал (для этого нужно проверить, что материал не встречался в true_train).

In [17]:
top_k = 10

# interactions['prediction_popular'] =
####### Здесь ваш код ##########
interactions['prediction_popular'] = (
    interactions.true_train
    .apply(
        lambda x:
        popular_content[~np.in1d(popular_content, x)][:top_k]
    )
)
################################
interactions['prediction_popular'][0]

array(['-6783772548752091658', '-133139342397538859',
       '-8208801367848627943', '8224860111193157980',
       '7507067965574797372', '-2358756719610361882',
       '-6843047699859121724', '-1297580205670251233',
       '8657408509986329668', '3367026768872537336'], dtype=object)

Настало время оценить качество. Посчитаем precision@10 для каждого пользователя (доля угаданных рекомендаций). Усредним по всем пользователям. Везде далее будем считать эту же метрику.

In [18]:
def calc_precision(column):
    ####### Здесь ваш код ##########
    return (
        interactions
        .apply(
            lambda row:
            len(set(row['true_test']).intersection(
                set(row[column]))) /
            min(len(row['true_test']) + 0.001, 10.0),
            axis=1)).mean()

    ################################

In [19]:
calc_precision('prediction_popular')

0.006454207722621084

## Часть 2. Коллаборативная фильтрация.

Перейдём к более сложному механизму рекомендаций, а именно коллаборативной фильтрации. Суть коллаборативной фильтрации в том, что учитывается схожесть пользователей и товаров между собой, а не факторы, которые их описывают. 

Для начала для удобства составим матрицу "оценок" пользователей. Нули будут обозначать отсутствие взаимодействия.

In [20]:
ratings = pd.pivot_table(
    interactions_train_df,
    values='eventStrength',
    index='personId',
    columns='contentId').fillna(0)

### Memory-based

Посчитаем схожести пользователей с помощью корреляции Пирсона. Для каждой пары учитываем только ненулевые значения.

Для скорости работы лучше переходить от pandas к numpy.

In [21]:
ratings_m = ratings.values

In [22]:
similarity_users = np.zeros((len(ratings_m), len(ratings_m)))

for i in tqdm_notebook(range(len(ratings_m)-1)):
    for j in range(i+1, len(ratings_m)):
        
        # nonzero elements of two users
        mask_uv = (ratings_m[i] != 0) & (ratings_m[j] != 0)
        
        # continue if no intersection
        if np.sum(mask_uv) == 0:
            continue
            
        # get nonzero elements
        ratings_v = ratings_m[i, mask_uv]
        ratings_u = ratings_m[j, mask_uv]
        
        # for nonzero std
        if len(np.unique(ratings_v)) < 2 or len(np.unique(ratings_u)) < 2:
            continue
        # similarity_users[i,j] = 
        # similarity_users[j,i] = 
        ####### Здесь ваш код ##########
        similarity_users[i,j] = np.corrcoef(ratings_v, ratings_u)[0, 1]
        similarity_users[j,i] = similarity_users[i,j]
        ################################

Теперь у нас есть матрицы схожести пользователей. Их можно использовать для рекомендаций.

Для каждого пользователя:

1. Найдём пользователей с похожестью больше $\alpha$ на нашего пользователя.
2. Посчитаем для каждой статьи долю пользователей (среди выделенных на первом шаге), которые взаимодействовали со статьёй.
3. Порекомендуем статьи с наибольшими долями со второго шага (среди тех, которые пользователь ещё не видел).

В нашем примере данных не очень много, поэтому возьмём $\alpha = 0$.

После того, как будут сделаны предсказания (новый столбец в interactions), посчитаем качество по той же метрике.

In [23]:
prediction_user_based = []
####### Здесь ваш код ##########
for i in tqdm_notebook(range(len(similarity_users))):
    users_sim = similarity_users[i] > 0
    if len(users_sim) == 0:
        prediction_user_based.append([])
    else:
        tmp_recommend = np.argsort(ratings_m[users_sim].sum(axis=0))[::-1]
        tmp_recommend = ratings.columns[tmp_recommend]
        recommend = np.array(tmp_recommend)[~np.in1d(tmp_recommend, interactions.iloc[i])][:10]
        prediction_user_based.append(list(recommend))
################################
interactions['prediction_user_based'] = prediction_user_based

In [24]:
calc_precision('prediction_user_based')

0.003541692918885617

### Модель со скрытыми переменными

Реализуем подход с разложением матрицы оценок. Для этого сделаем сингулярное разложение (svd в scipy.linalg), на выходе вы получите три матрицы.

Заметим, что мы используем матрицу с нулями, будто отсутствующие взаимодействия негативные, что странно.

Если бы мы учили модель со скрытыми переменными с помощью стохастического градиентного спуска, то неизвестные взаимодействия могли бы не использовать.

In [25]:
from scipy.linalg import svd

In [26]:
####### Здесь ваш код ##########
U, sigma, V = svd(ratings)
print(ratings.shape, U.shape, sigma.shape, V.shape)
################################

(1112, 2366) (1112, 1112) (1112,) (2366, 2366)


In [27]:
####### Здесь ваш код ##########
Sigma = np.zeros((1112, 2366))
Sigma[:1112, :1112] = np.diag(sigma)

new_ratings = U.dot(Sigma).dot(V)
print(sum(sum((new_ratings - ratings.values) ** 2)))
################################

8.464877207035405e-25


Значения у матрицы с сингулярными числами отсортированы по убыванию. Допустим мы хотим оставить только первые 100 компонент (и получить скрытые представления размерности 100). Для этого необходимо оставить 100 столбцов в матрице U, оставить из sigma только первые 100 значений (и сделать из них диагональную матрицу) и 100 столбцов в матрице V. Перемножим преобразованные матрицы ($\hat{U}, \hat{sigma}, \hat{V^T}$), чтобы получить восстановленную матрицу оценок.

In [28]:
K = 100
####### Здесь ваш код ##########
sigma[K:] = 0
Sigma = np.zeros((1112, 2366))
Sigma[:1112, :1112] = np.diag(sigma)
################################

Посчитаем качество аппроксимации матрицы по норме Фробениуса (среднеквадратичную ошибку между всеми элементами соответствующими элементами двух матриц). Сравним его с простым бейзлайном с константным значением, равным среднему значению исходной матрицы. У аппроксимации ошибка должна получиться ниже.

In [29]:
####### Здесь ваш код ##########
new_ratings = U.dot(Sigma).dot(V)
print(sum(sum((new_ratings - ratings.values) ** 2)))
print(sum(sum((ratings.values.mean() - ratings.values) ** 2)))
################################

25843.424898912766
78622.13000466056


Теперь можно делать предсказания по матрице. Сделаем их (не забываем про то, что уже было просмотрено пользователем), оценим качество. Для этого необходимо для каждого пользователя найти предметы с наибольшими оценками в восстановленной матрице.

In [30]:
new_ratings = pd.DataFrame(new_ratings, index=ratings.index, columns=ratings.columns)

predictions = []
####### Здесь ваш код ##########
for personId in tqdm_notebook(interactions.index):
    prediction = (
        new_ratings
        .loc[personId]
        .sort_values(ascending=False)
        .index.values
    )
    
    predictions.append(
        list(prediction[~np.in1d(
            prediction,
            interactions.loc[personId, 'true_train'])])[:top_k])
################################
interactions['prediction_svd'] = predictions

In [31]:
calc_precision('prediction_svd')

0.012212989310270754

## Часть 3. Контентные  модели

В этой части реализуем альтернативных подход к рекомендательным системам — контентные модели.

Теперь мы будем оперировать не матрицей с оценками, а классической для машинного обучения матрицей объекты-признаки. Каждый объект будет характеризовать пару user-item и содержать признаки, описывающие как пользователя, так и товар. Кроме этого признаки могут описывать и саму пару целиком.

Матрица со всеми взаимодействиями уже получена нами на этапа разбиения выборки на 2 части. 

Будем обучать классификатор на взаимодействие, а для него нужны отрицательные примеры. Добавим случайные отсутствующие взаимодействия как отрицательные.

Заметим, что модель оценивает каждую пару потенциального взаимодействия, а значит, надо подготовить выборку из всех возможных пар из пользователей и статей.

In [32]:
test_personId = np.repeat(interactions.index, len(ratings.columns)) 
test_contentId = list(ratings.columns) * len(interactions)
test = pd.DataFrame(
    np.array([test_personId, test_contentId]).T,
    columns=['personId', 'contentId'])

interactions_train_df = pd.concat((
    interactions_train_df,
    test.loc[
        np.random.permutation(test.index)[
            :4*len(interactions_train_df)]]), ignore_index=True)
interactions_train_df.eventStrength.fillna(0, inplace=True)

Придумаем и добавим признаков о пользователях и статьях. Сначала добавим информацию о статьях в данные о взаимодействиях.

In [33]:
interactions_train_df = interactions_train_df.merge(articles_df, how='left', on='contentId')
interactions_test_df = interactions_test_df.merge(articles_df, how='left', on='contentId')

In [34]:
# first feature index
features_start = len(interactions_train_df.columns)

После обучения модели нам придётся делать предсказания на тестовой выборке для всех возможных пар статья-пользователь. Подготовим такую матрицу, чтобы параллельно посчитать признаки для неё.

In [35]:
test_personId = np.repeat(interactions.index, len(articles_df)) 
test_contentId = list(articles_df.contentId) * len(interactions)
test = pd.DataFrame(
    np.array([test_personId, test_contentId]).T,
    columns=['personId', 'contentId'])
test = test.merge(articles_df, how='left', on='contentId')

test.head()

,personId,contentId,timestamp,eventType,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,-1007001694607905623,-4110354420726924665,1459193988,CONTENT SHARED,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,-1007001694607905623,-7292285110016212249,1459194146,CONTENT SHARED,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
2,-1007001694607905623,-6151852268067518688,1459194474,CONTENT SHARED,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
3,-1007001694607905623,2448026894306402386,1459194497,CONTENT SHARED,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en
4,-1007001694607905623,-2826566343807132236,1459194522,CONTENT SHARED,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.coindesk.com/ieee-blockchain-oxford...,IEEE to Talk Blockchain at Cloud Computing Oxf...,One of the largest and oldest organizations fo...,en


Добавим признаки-индикаторы возможных значений contentType.

In [36]:
interactions_train_df['is_HTML'] = interactions_train_df.contentType == 'HTML'
####### Здесь ваш код ##########
interactions_train_df['is_RICH'] = interactions_train_df.contentType == 'RICH'
interactions_train_df['is_VIDEO'] = interactions_train_df.contentType == 'VIDEO'

test['is_HTML'] = test.contentType == 'HTML'
test['is_RICH'] = test.contentType == 'RICH'
test['is_VIDEO'] = test.contentType == 'VIDEO'
################################

Добавим признаки "длина названия" и "длина текста" + некоторые проверки на ключевые слова.

In [37]:
interactions_train_df['title_length'] = interactions_train_df.title.fillna('').apply(len)
####### Здесь ваш код ##########
interactions_train_df['text_length'] = interactions_train_df.text.fillna('').apply(len)

test['title_length'] = test.title.fillna('').apply(len)
test['text_length'] = test.text.fillna('').apply(len)

interactions_train_df['has_new'] = \
    interactions_train_df.title.fillna('').apply(lambda x: 'new' in x.lower())
interactions_train_df['has_why'] = \
    interactions_train_df.title.fillna('').apply(lambda x: 'why' in x.lower())
interactions_train_df['has_how'] = \
    interactions_train_df.title.fillna('').apply(lambda x: 'how' in x.lower())
interactions_train_df['has_ai'] = \
    interactions_train_df.title.fillna('').apply(lambda x: 'ai' in x.lower())

test['has_new'] = \
    test.title.fillna('').apply(lambda x: 'new' in x.lower())
test['has_why'] = \
    test.title.fillna('').apply(lambda x: 'why' in x.lower())
test['has_how'] = \
    test.title.fillna('').apply(lambda x: 'how' in x.lower())
test['has_ai'] = \
    test.title.fillna('').apply(lambda x: 'ai' in x.lower())
################################

Добавим признаки-индикаторы языка.

In [38]:
interactions_train_df['is_lang_en'] = interactions_train_df.lang == 'en'
####### Здесь ваш код ##########
interactions_train_df['is_lang_pt'] = interactions_train_df.lang == 'pt'

test['is_lang_en'] = test.lang == 'en'
test['is_lang_pt'] = test.lang == 'pt'
################################

Обучим на полученных признаках градиентный бустинг.

In [39]:
import catboost 

model = catboost.CatBoostClassifier()
model.fit(interactions_train_df[interactions_train_df.columns[features_start:]],
          np.array(interactions_train_df.eventStrength > 0, dtype=int))

0:	learn: 0.6644216	total: 231ms	remaining: 3m 50s
1:	learn: 0.6397517	total: 282ms	remaining: 2m 20s
2:	learn: 0.6187005	total: 345ms	remaining: 1m 54s
3:	learn: 0.6006784	total: 393ms	remaining: 1m 37s
4:	learn: 0.5851678	total: 453ms	remaining: 1m 30s
5:	learn: 0.5719286	total: 506ms	remaining: 1m 23s
6:	learn: 0.5605556	total: 561ms	remaining: 1m 19s
7:	learn: 0.5508589	total: 617ms	remaining: 1m 16s
8:	learn: 0.5425047	total: 667ms	remaining: 1m 13s
9:	learn: 0.5353031	total: 732ms	remaining: 1m 12s
10:	learn: 0.5291311	total: 784ms	remaining: 1m 10s
11:	learn: 0.5239659	total: 851ms	remaining: 1m 10s
12:	learn: 0.5193952	total: 911ms	remaining: 1m 9s
13:	learn: 0.5154735	total: 967ms	remaining: 1m 8s
14:	learn: 0.5121020	total: 1.02s	remaining: 1m 7s
15:	learn: 0.5091228	total: 1.07s	remaining: 1m 5s
16:	learn: 0.5066393	total: 1.12s	remaining: 1m 4s
17:	learn: 0.5043147	total: 1.17s	remaining: 1m 3s
18:	learn: 0.5023287	total: 1.22s	remaining: 1m 3s
19:	learn: 0.5006215	total: 1

161:	learn: 0.4770537	total: 8.02s	remaining: 41.5s
162:	learn: 0.4769833	total: 8.06s	remaining: 41.4s
163:	learn: 0.4769159	total: 8.1s	remaining: 41.3s
164:	learn: 0.4768493	total: 8.13s	remaining: 41.1s
165:	learn: 0.4767954	total: 8.16s	remaining: 41s
166:	learn: 0.4767169	total: 8.21s	remaining: 41s
167:	learn: 0.4766601	total: 8.25s	remaining: 40.9s
168:	learn: 0.4766359	total: 8.28s	remaining: 40.7s
169:	learn: 0.4765771	total: 8.32s	remaining: 40.6s
170:	learn: 0.4765404	total: 8.38s	remaining: 40.6s
171:	learn: 0.4764443	total: 8.46s	remaining: 40.7s
172:	learn: 0.4763494	total: 8.52s	remaining: 40.7s
173:	learn: 0.4762866	total: 8.6s	remaining: 40.8s
174:	learn: 0.4762108	total: 8.67s	remaining: 40.9s
175:	learn: 0.4761361	total: 8.74s	remaining: 40.9s
176:	learn: 0.4760673	total: 8.79s	remaining: 40.9s
177:	learn: 0.4760175	total: 8.83s	remaining: 40.8s
178:	learn: 0.4759300	total: 8.87s	remaining: 40.7s
179:	learn: 0.4758410	total: 8.93s	remaining: 40.7s
180:	learn: 0.4757

320:	learn: 0.4686459	total: 15.3s	remaining: 32.4s
321:	learn: 0.4686100	total: 15.4s	remaining: 32.4s
322:	learn: 0.4685912	total: 15.4s	remaining: 32.3s
323:	learn: 0.4685357	total: 15.5s	remaining: 32.3s
324:	learn: 0.4684726	total: 15.5s	remaining: 32.3s
325:	learn: 0.4684443	total: 15.6s	remaining: 32.2s
326:	learn: 0.4684053	total: 15.6s	remaining: 32.2s
327:	learn: 0.4683636	total: 15.7s	remaining: 32.1s
328:	learn: 0.4683313	total: 15.7s	remaining: 32.1s
329:	learn: 0.4682710	total: 15.8s	remaining: 32s
330:	learn: 0.4682155	total: 15.8s	remaining: 31.9s
331:	learn: 0.4681807	total: 15.8s	remaining: 31.9s
332:	learn: 0.4681549	total: 15.9s	remaining: 31.8s
333:	learn: 0.4681266	total: 15.9s	remaining: 31.8s
334:	learn: 0.4680756	total: 16s	remaining: 31.8s
335:	learn: 0.4680472	total: 16.1s	remaining: 31.8s
336:	learn: 0.4680072	total: 16.1s	remaining: 31.7s
337:	learn: 0.4680029	total: 16.2s	remaining: 31.7s
338:	learn: 0.4678952	total: 16.2s	remaining: 31.6s
339:	learn: 0.46

479:	learn: 0.4625669	total: 22.6s	remaining: 24.5s
480:	learn: 0.4625293	total: 22.6s	remaining: 24.4s
481:	learn: 0.4624825	total: 22.7s	remaining: 24.4s
482:	learn: 0.4624462	total: 22.7s	remaining: 24.3s
483:	learn: 0.4624345	total: 22.7s	remaining: 24.2s
484:	learn: 0.4624098	total: 22.8s	remaining: 24.2s
485:	learn: 0.4623734	total: 22.8s	remaining: 24.1s
486:	learn: 0.4623266	total: 22.9s	remaining: 24.1s
487:	learn: 0.4623143	total: 22.9s	remaining: 24s
488:	learn: 0.4622937	total: 23s	remaining: 24s
489:	learn: 0.4622586	total: 23s	remaining: 23.9s
490:	learn: 0.4622372	total: 23.1s	remaining: 23.9s
491:	learn: 0.4622195	total: 23.1s	remaining: 23.8s
492:	learn: 0.4621952	total: 23.1s	remaining: 23.8s
493:	learn: 0.4621611	total: 23.2s	remaining: 23.7s
494:	learn: 0.4621519	total: 23.2s	remaining: 23.7s
495:	learn: 0.4621310	total: 23.3s	remaining: 23.7s
496:	learn: 0.4621200	total: 23.4s	remaining: 23.6s
497:	learn: 0.4621073	total: 23.4s	remaining: 23.6s
498:	learn: 0.462076

638:	learn: 0.4588349	total: 31.2s	remaining: 17.6s
639:	learn: 0.4588188	total: 31.2s	remaining: 17.6s
640:	learn: 0.4587997	total: 31.3s	remaining: 17.5s
641:	learn: 0.4587901	total: 31.4s	remaining: 17.5s
642:	learn: 0.4587686	total: 31.5s	remaining: 17.5s
643:	learn: 0.4587556	total: 31.5s	remaining: 17.4s
644:	learn: 0.4587328	total: 31.6s	remaining: 17.4s
645:	learn: 0.4587225	total: 31.6s	remaining: 17.3s
646:	learn: 0.4587089	total: 31.7s	remaining: 17.3s
647:	learn: 0.4586982	total: 31.7s	remaining: 17.2s
648:	learn: 0.4586872	total: 31.8s	remaining: 17.2s
649:	learn: 0.4586798	total: 31.8s	remaining: 17.1s
650:	learn: 0.4586303	total: 31.9s	remaining: 17.1s
651:	learn: 0.4585739	total: 31.9s	remaining: 17s
652:	learn: 0.4585573	total: 32s	remaining: 17s
653:	learn: 0.4585426	total: 32.1s	remaining: 17s
654:	learn: 0.4585186	total: 32.1s	remaining: 16.9s
655:	learn: 0.4585082	total: 32.2s	remaining: 16.9s
656:	learn: 0.4584751	total: 32.2s	remaining: 16.8s
657:	learn: 0.458459

797:	learn: 0.4556637	total: 40s	remaining: 10.1s
798:	learn: 0.4556344	total: 40.1s	remaining: 10.1s
799:	learn: 0.4556295	total: 40.1s	remaining: 10s
800:	learn: 0.4556068	total: 40.2s	remaining: 9.98s
801:	learn: 0.4555973	total: 40.2s	remaining: 9.93s
802:	learn: 0.4555870	total: 40.2s	remaining: 9.87s
803:	learn: 0.4555583	total: 40.3s	remaining: 9.82s
804:	learn: 0.4555263	total: 40.3s	remaining: 9.77s
805:	learn: 0.4555134	total: 40.4s	remaining: 9.72s
806:	learn: 0.4554807	total: 40.4s	remaining: 9.67s
807:	learn: 0.4554720	total: 40.5s	remaining: 9.62s
808:	learn: 0.4554296	total: 40.5s	remaining: 9.57s
809:	learn: 0.4554147	total: 40.6s	remaining: 9.52s
810:	learn: 0.4553677	total: 40.6s	remaining: 9.47s
811:	learn: 0.4553568	total: 40.7s	remaining: 9.42s
812:	learn: 0.4553217	total: 40.7s	remaining: 9.37s
813:	learn: 0.4553155	total: 40.8s	remaining: 9.32s
814:	learn: 0.4552833	total: 40.8s	remaining: 9.27s
815:	learn: 0.4552671	total: 40.9s	remaining: 9.22s
816:	learn: 0.45

956:	learn: 0.4530386	total: 47s	remaining: 2.11s
957:	learn: 0.4530246	total: 47.1s	remaining: 2.06s
958:	learn: 0.4530160	total: 47.1s	remaining: 2.01s
959:	learn: 0.4530004	total: 47.2s	remaining: 1.97s
960:	learn: 0.4529758	total: 47.2s	remaining: 1.92s
961:	learn: 0.4529669	total: 47.3s	remaining: 1.87s
962:	learn: 0.4529505	total: 47.3s	remaining: 1.82s
963:	learn: 0.4529238	total: 47.4s	remaining: 1.77s
964:	learn: 0.4528951	total: 47.4s	remaining: 1.72s
965:	learn: 0.4528840	total: 47.4s	remaining: 1.67s
966:	learn: 0.4528735	total: 47.5s	remaining: 1.62s
967:	learn: 0.4528707	total: 47.5s	remaining: 1.57s
968:	learn: 0.4528614	total: 47.6s	remaining: 1.52s
969:	learn: 0.4528585	total: 47.6s	remaining: 1.47s
970:	learn: 0.4528401	total: 47.6s	remaining: 1.42s
971:	learn: 0.4528012	total: 47.7s	remaining: 1.37s
972:	learn: 0.4527911	total: 47.7s	remaining: 1.32s
973:	learn: 0.4527851	total: 47.8s	remaining: 1.27s
974:	learn: 0.4527754	total: 47.8s	remaining: 1.23s
975:	learn: 0.

Сделаем предсказания на тестовой выборке, сформируем из них рекомендации. 

In [40]:
####### Здесь ваш код ##########
predictions = model.predict_proba(test[interactions_train_df.columns[features_start:]])[:, 1]
test['predictions'] = predictions

test = test.sort_values('predictions', ascending=False)
predictions = test.groupby('personId')['contentId'].aggregate(list)
tmp_predictions = []

for personId in tqdm_notebook(interactions.index):
    prediction = np.array(predictions.loc[personId])
    
    tmp_predictions.append(
        list(prediction[~np.in1d(
            prediction,
            interactions.loc[personId, 'true_train'])])[:top_k])
    
interactions['prediction_content'] = tmp_predictions
################################

Оценим их качество.

In [41]:
# calc_precision(...)
####### Здесь ваш код ##########
calc_precision('prediction_content')
################################

0.007991753561699618

## Часть 4. Факторизационная машина

Вспомним, что факторизационная машина учитывает попарные взаимодействия признаков, что приводит сразу и к использованию контента (сами признаки), и к обучению скрытых представлений (индикаторы пользователей и статей).

Попробуем факторизационные машины из библиотеки pyFM (так как можно работать прямо из питона). https://github.com/coreylynch/pyFM

In [42]:
from pyfm import pylibfm
from sklearn.feature_extraction import DictVectorizer

ModuleNotFoundError: No module named 'pyfm'

Перейдём к обобщению матричных разложений — факторизационным машинам, которые работают могут работать с контентной информацией. Вспомним, какие данные у нас изначально были:

В факторизационную машину можно загрузить "айдишники" пользователей и статей (то есть сделать аналог коллаборативной фильтрации) и одновременно различные признаки.

Удобно обрабатывать категориальные переменные (id и другие) можно с помощью DictVectorizer. Например, процесс может выглядить вот так:
```python
train = [
    {"user": "1", "item": "5", "age": 19},
    {"user": "2", "item": "43", "age": 33},
    {"user": "3", "item": "20", "age": 55},
    {"user": "4", "item": "10", "age": 20},
]
v = DictVectorizer()
X = v.fit_transform(train)
y = np.repeat(1.0, X.shape[0])
fm = pylibfm.FM()
fm.fit(X,y)
fm.predict(v.transform({"user": "1", "item": "10", "age": 24}))
```

Сгенерируем таблицу с признаками в таком виде, где будут id пользователя, статьи и автора статьи и несколько признаков, которые вы сможете придумать.

In [311]:
train_data = []

for i in tqdm_notebook(range(len(interactions_train_df))):
    features = {}
    features['personId'] = str(interactions_train_df.iloc[i].personId)
    features['contentId'] = str(interactions_train_df.iloc[i].contentId)
    ####### Здесь ваш код ##########
    try:
        article = articles_df.loc[features['contentId']]
        features['authorId'] = str(article.authorPersonId)
        features['authorCountry'] = str(article.authorCountry)
        features['lang'] = str(article.lang)
    except:
        features['authorId'] = 'unknown'
        features['authorCountry'] = 'unknown'
        features['lang'] = 'unknown'
    ################################
    train_data.append(features)

Повторим эту процедуру для тестовой выборки. Заметим, что модель оценивает каждую пару потенциального взаимодействия, а значит, надо подготовить выборку из всех возможных пар из пользователей и статей.

In [312]:
from copy import deepcopy

test_data = []

for i in tqdm_notebook(range(len(interactions))):
    features = {}
    features['personId'] = str(interactions.index[i])  
    for j in range(len(ratings.columns)):
        ####### Здесь ваш код ##########
        features['contentId'] = str(ratings.columns[j])
        
        try:
            article = articles_df.loc[features['contentId']]
            features['authorId'] = str(article.authorPersonId)
            features['authorCountry'] = str(article.authorCountry)
            features['lang'] = str(article.lang)
        except:
            features['authorId'] = 'unknown'
            features['authorCountry'] = 'unknown'
            features['lang'] = 'unknown'
        ################################
        test_data.append(deepcopy(features))

Векторизуем, получим разреженные матрицы.

Мы будем обучать регрессор на силу взаимодействия, а для него нужны отрицательные примеры. Добавим некоторое количество случайных примеров как негативные (матрица взаимодействий разреженная, поэтому шансы взять как негативное взаимодействие некоторое положительное мало).

In [313]:
dv = DictVectorizer()

train_features = dv.fit_transform(
    train_data + list(np.random.permutation(test_data)[:100000]))
test_features = dv.transform(test_data)

In [314]:
train_features

In [315]:
y_train = list(interactions_train_df.eventStrength.values) + list(np.zeros(100000))

Укажем размером скрытого представления 10, сделаем 10 итераций.

In [316]:
# fm = pylibfm.FM(...
####### Здесь ваш код ##########
fm = pylibfm.FM(num_factors=10, num_iter=30, task='regression')
################################
fm.fit(train_features, y_train)

Предскажем и оценим качество.

In [317]:
####### Здесь ваш код ##########
test_features = dv.transform(test_data)

y_predict = fm.predict(test_features)

new_ratings = y_predict.reshape((1112, 2366))
################################

In [320]:
predictions = []

for i, person in enumerate(interactions.index):
    user_prediction = ratings.columns[np.argsort(new_ratings[i])[::-1]]
    predictions.append(
        user_prediction[~np.in1d(user_prediction,
                                 interactions.loc[person, 'true_train'])][:top_k])
    
interactions['fm_prediction'] = predictions

In [321]:
calc_precision('fm_prediction')